# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow instructions and use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever,
to query against two pdf documents we will upload.

The architeture and data flow diagram below depicts the interaction among all components that comprise OpenAI Assistant APIs. Central to understand is the Threads and Runtime that executes anyschronously, adding and reading messages to the Threads.

For integrating the Assistants API:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools

Below we follow those steps to demonstrate how to integrate Assistants API, using function tool, to ask our Assistant to solve simple and complex Maths problems.

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="./images/assistant_ai_tools_functions.png">


## How to use Assistant API using Tools: Function calling

In [1]:
import warnings
import os
import json

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
from typing import List, Dict, Any
from assistant_utils import print_thread_messages, upload_files, \
                            loop_until_completed, create_assistant_run
from function_utils import add_prime_numbers

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Assistant API calling for Anyscale Endpoints (which serves only OS modles) is not yet aviable).

In [2]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

### Step 1: Create our custom function definition
This our JSON object definiton for our function:
* name of the funciton
* parameters for the funtion
* type of arguments
* descriptions for function and each parameter type

In [4]:
sum_of_prime_numbers = {
        "name": "add_prime_numbers",
        "description": "Add a list of 25 integer prime numbers between 2 and 100",
        "parameters": {
            "type": "object",
            "properties": {
                "prime_numbers": {
                    "type": "array",
                    "items": {
                    "type": "integer",
                        "description": "A integer list of 25 random prime numbers betwee 2 and 100"
                         },
                        "description": "List of of 25 prime numbers to be added"
                    }
                },
                "required": ["add_prime_numbers"]
            }
        }

tools = [{'type': 'function', 'function': sum_of_prime_numbers}]

### Step 2: Create an Assistant 
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object. Supply the Assistant with a model to use, tools, i.e., functions

In [5]:
assistant = client.beta.assistants.create(name="AI Math Tutor",
                                           instructions="""You are a knowledgeable chatbot trained to help 
                                               solve basic and advanced grade 8-12 Maths problems.
                                               Use a neutral, teacher and  advisory tone""",
                                           model=MODEL,
                                           tools=tools)
assistant

Assistant(id='asst_gxX3X03ih8AuDTUZhKblIHQx', created_at=1703299773, description=None, file_ids=[], instructions='You are a knowledgeable chatbot trained to help \n                                               solve basic and advanced grade 8-12 Maths problems.\n                                               Use a neutral, teacher and  advisory tone', metadata={}, model='gpt-4-1106-preview', name='AI Math Tutor', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='add_prime_numbers', description='Add a list of 25 integer prime numbers between 2 and 100', parameters={'type': 'object', 'properties': {'prime_numbers': {'type': 'array', 'items': {'type': 'integer', 'description': 'A integer list of 25 random prime numbers betwee 2 and 100'}, 'description': 'List of of 25 prime numbers to be added'}}, 'required': ['add_prime_numbers']}), type='function')])

### Step 3: Create an empty thread 
As the diagram above shows, the Thread is the object with which the AI Assistant runs will interact with, by fetching messages and putting messages to it. Think of a thread as a "conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [6]:
thread = client.beta.threads.create()
thread

Thread(id='thread_x7NYYLSPdNqCVip9IwGSwAmI', created_at=1703299781, metadata={}, object='thread')

### Step 4: Add your message query to the thread for the Assistant

In [7]:
message_1 = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""Generate 25 random prime numbers between 2 and 100
    as a list, and add the numbers in this generated list
    """
)
message_1

ThreadMessage(id='msg_9HHbaRdRGzlX5opvscRDuxw1', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Generate 25 random prime numbers between 2 and 100\n    as a list, and add the numbers in this generated list\n    '), type='text')], created_at=1703299831, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_x7NYYLSPdNqCVip9IwGSwAmI')

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses it’s configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, requires, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [8]:
instruction_msg = """Generate a random list of 25 prime numbers between 
2 and 100."""
run_1 = create_assistant_run(client, assistant, thread, instruction_msg)
run_1

Run(id='run_pbol5shDICiYjAlxS4XYBuBH', assistant_id='asst_gxX3X03ih8AuDTUZhKblIHQx', cancelled_at=None, completed_at=None, created_at=1703299861, expires_at=1703300461, failed_at=None, file_ids=[], instructions='Generate a random list of 25 prime numbers between \n2 and 100.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_x7NYYLSPdNqCVip9IwGSwAmI', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add_prime_numbers', description='Add a list of 25 integer prime numbers between 2 and 100', parameters={'type': 'object', 'properties': {'prime_numbers': {'type': 'array', 'items': {'type': 'integer', 'description': 'A integer list of 25 random prime numbers betwee 2 and 100'}, 'description': 'List of of 25 prime numbers to be added'}}, 'required': ['add_prime_numbers']}), type='function')])

### Step 6: Retrieve the status

In [9]:
run_1_obj = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run_1.id
)

In [10]:
print(run_1_obj, run_1_obj.status)

Run(id='run_pbol5shDICiYjAlxS4XYBuBH', assistant_id='asst_gxX3X03ih8AuDTUZhKblIHQx', cancelled_at=None, completed_at=None, created_at=1703299861, expires_at=1703300461, failed_at=None, file_ids=[], instructions='Generate a random list of 25 prime numbers between \n2 and 100.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_gXieuNHDecF2voSIfXOIYEe4', function=Function(arguments='{"prime_numbers":[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]}', name='add_prime_numbers'), type='function')]), type='submit_tool_outputs'), started_at=1703299861, status='requires_action', thread_id='thread_x7NYYLSPdNqCVip9IwGSwAmI', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add_prime_numbers', description='Add a list of 25 integer prime numbers between 2 and 100', parameters={'type': 'object',

Loop until run status is "required_action," which is a trigger notification to extract arguments generated by the LLM model and 
carry onto the next step: invoke the function with the generated arguments.

In [11]:
loop_until_completed(client, thread, run_1_obj)

Get the tools output. This where your application can call into
the funtion you want invoked. 

### Step 7: Invoke our custom code function

In [12]:
def get_output_for_tool_call(tool_call: List[Any]) -> object:
    for each_tool in tool_calls:
        tool_call_id = each_tool.id
        function_args = json.loads(each_tool.function.arguments)
        print(f"Tool ID: {tool_call_id}")
        print(f"Function args:{function_args}")
        # invoke your function here and return the
        # output back to the Assistant so it can format
        # the final message
        sum_of_primes = add_prime_numbers(function_args)
        return [{ "tool_call_id": tool_call_id,
                 "output": f"The sum of random prime numbers between 2 and 100 is {sum_of_primes}"
               }]
    

In [13]:
tool_calls = run_1_obj.required_action.submit_tool_outputs.tool_calls
tool_calls

[RequiredActionFunctionToolCall(id='call_gXieuNHDecF2voSIfXOIYEe4', function=Function(arguments='{"prime_numbers":[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]}', name='add_prime_numbers'), type='function')]

In [14]:
tool_outputs = get_output_for_tool_call(tool_calls)
tool_outputs

Tool ID: call_gXieuNHDecF2voSIfXOIYEe4
Function args:{'prime_numbers': [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]}


[{'tool_call_id': 'call_gXieuNHDecF2voSIfXOIYEe4',
  'output': 'The sum of random prime numbers between 2 and 100 is 1060'}]

### Step 7: Create the final run to finish.
Submit the output from invoked funcition and pass it back
to the Assistant to generated the final message for the user

In [15]:
run_2_obj = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run_1_obj.id,
    tool_outputs=tool_outputs
)

In [16]:
loop_until_completed(client, thread, run_2_obj)

in_progress
completed


In [17]:
print_thread_messages(client, thread)

('assistant:The sum of the random list of 25 prime numbers between 2 and 100 '
 'is 1060.')
('user:Generate 25 random prime numbers between 2 and 100\n'
 '    as a list, and add the numbers in this generated list\n'
 '    ')


In [18]:
# Delete the assistant. 
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_gxX3X03ih8AuDTUZhKblIHQx', deleted=True, object='assistant.deleted')
